In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [2]:
# 整理ytd表格
ytd = pd.read_excel(r'D:/Python_project/INPAQ_price_comparison/YTD Demand 9-28.xlsx')
# ytd = ytd.columns = ['Supplier PN', 'YTD till Sep']
ytd = ytd.rename(columns={'Item':'Supplier PN'})
# ytd


In [3]:
#整理報價記錄
quotation_record = pd.read_excel(r'D:\Python_project\INPAQ_price_comparison/Price Quote list Cathy 20200929_python.xlsx')
filter = (quotation_record['Customer'] == 'Laird')
quotation_record = quotation_record[filter]
# quotation_record


In [4]:
# 有報過價格的記錄
matched_quotation = pd.merge(quotation_record, ytd, on=['Supplier PN'], how='right')
matched_quotation = matched_quotation.loc[:,['Supplier PN', 'Inpaq PN', 'Date', 'Inpaq Price(USD/Kpcs)','YTD till Sep']]
matched_quotation['Inpaq Price(USD/Kpcs)'] = matched_quotation['Inpaq Price(USD/Kpcs)'].map(lambda x: x * 1000 if x < 0.01 else x)
matched_quotation = matched_quotation.drop_duplicates(subset=['Supplier PN'], keep="first")
# matched_quotation['YTD till Sep']
# matched_quotation.to_excel('test.xlsx', index=False)


In [5]:
shanghai_price = pd.read_excel(r'D:\Python_project\INPAQ_price_comparison\順德報價20191205.xlsx')
final_table = pd.merge(matched_quotation, shanghai_price, on=['Supplier PN'], how='left')
# final_table

In [6]:
zhunan_price = pd.read_excel(r'D:\Python_project\INPAQ_price_comparison\出貨明細查找.xlsx')
# zhunan_price
final_table = pd.merge(final_table, zhunan_price, on=['Supplier PN'], how='left')
# final_table

In [7]:
# final_table

In [8]:
robin_quotation = pd.read_excel(r'D:\Python_project\INPAQ_price_comparison\robin_quotation.xlsx')
table_with_robin = pd.merge(final_table, robin_quotation, on=['Inpaq PN'], how='left')
table_with_robin = table_with_robin.loc[:,['Date', 'Supplier PN', 'Inpaq PN_x','Inpaq Price(USD/Kpcs)','Inpaq PN_y','不含税出厂单价\n（元/个）','Inpaq PN','出貨明細價格','Robin Quotation','Shipment','YTD till Sep']]
table_with_robin
final_version = table_with_robin.rename(columns={'Inpaq Price(USD/Kpcs)':'Cathy報價記錄','不含税出厂单价\n（元/个）':'順德報價','Inpaq PN_x':'報價記錄PN','Inpaq PN_y':'順德報價PN','Inpaq PN':'出貨明細PN'})
ian_table = pd.read_excel(r'D:\Python_project\INPAQ_price_comparison\ian_YTD.xlsx')
ian_table = ian_table.rename(columns={'Item':'Supplier PN', 'Unnamed: 2':'Inpaq PN'})
final_version = pd.merge(final_version, ian_table, on=['Supplier PN'], how='left').loc[:,['Date','Supplier PN','報價記錄PN','Cathy報價記錄','順德報價PN','順德報價','出貨明細PN','出貨明細價格','Robin Quotation','Shipment','YTD till Sep_x','Inpaq PN']]
final_version = final_version.rename(columns={'YTD till Sep_x':'YTD till Sep','Inpaq PN':'Ian對的PN'})
final_version.to_excel('laird_table.xlsx', index=False, sheet_name='Laird table')